# In progress: TMVA BDT classification 
Modifying code from https://github.com/arodel21/Boosted_Di-Higgs_Identification/blob/master/notebooks/Di-Higgs/Testing_BDT_New.ipynb xml models using current datasets

In [2]:
import ROOT
from ROOT import TMVA, TTree
from ROOT import TCanvas, TColor, TGaxis, TH1F, TPad

ROOT.TMVA.Tools.Instance()
## For PYMVA methods
TMVA.PyMethodBase.PyInitialize()

import itertools

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

from array import array
from root_numpy import root2array, tree2array
import numpy as np
import pandas as pd

from IPython.display import Image, display

## Path of output files generated by BDT classification

In [3]:
outputDir = "../bdt_output/"

In [4]:
def filter_region(file, region, tag, signal, output_dir):
    oldfile = ROOT.TFile(file)
    print("Reading from:", file)
    oldtree = oldfile.Nominal
    file_name = region+ "_" + str(tag) +"_"+signal
    signal_file = ROOT.TFile(output_dir + file_name+"_s.root","recreate")
    signal_tree = oldtree.CloneTree(0)
    backg_file = ROOT.TFile(output_dir + file_name+"_b.root","recreate")
    backg_tree = oldtree.CloneTree(0)
    data_file = ROOT.TFile(output_dir + file_name+"_d.root","recreate")
    data_tree = oldtree.CloneTree(0)
    flag=True
    for entry in oldtree:
        if (entry.m_region == region) and (entry.m_FJNbtagJets==tag):
            if (entry.sample == "data"):
                data_tree.Fill()
            elif (entry.sample == signal): #signal
                signal_tree.Fill()
            else:
                backg_tree.Fill()
    signal_tree.AutoSave()   
    backg_tree.AutoSave()
    data_tree.AutoSave()
    return {"s_t":signal_tree, "s_f":signal_file, 
            "b_t":backg_tree, "b_f":backg_file, 
            "d_t":data_tree, "d_f":data_file}

In [5]:
def get_params(file):
    l = []     
    with open(file, 'r') as f:
        for line in f:
            print(line)
            s = line.strip()
            l.append(s)
    
    return l

In [6]:
# For the BDT classification using TMVA we need separated signal and 
# background root files
d = filter_region("../processed_data/all_1000.root", "SR" , 
                  1, "Xtohh1000_Hw", outputDir)
signal_file = d["s_f"]
backg_file = d["b_f"] 

Reading from: ../processed_data/all_1000.root


In [7]:
def BDT(params, training, config, od, comb=''):
    
    signal_file = od + config+"_s.root"
    backg_file = od+config+"_b.root"
    output_file = od + config+"_"+comb+"_BDT_Classification.root"
    
    signal_input = ROOT.TFile(signal_file)
    signal_tree = signal_input.Nominal
    
    backg_input = ROOT.TFile(backg_file)
    backg_tree = backg_input.Nominal
    
    outputFile = ROOT.TFile.Open(output_file, "RECREATE")

    # Factory
    fact_string = "!V:ROC:Silent:Color:!DrawProgressBar:AnalysisType=Classification" 
    factory = ROOT.TMVA.Factory("config"+"_"+comb, outputFile,fact_string)


    loader = ROOT.TMVA.DataLoader("../bdt_output/dataset2")

    ### global event weights per tree (see below for setting event-wise weights)
    #signalWeight     = 1.0
    #backgroundWeight = 1.0

    ### You can add an arbitrary number of signal or background trees
    loader.AddSignalTree(signal_tree )
    loader.AddBackgroundTree(backg_tree )
    loader.SetWeightExpression("EventWeight")

    not_cons = ['sample', 'EventWeight', 'EventNumber', 
                'm_region', 'm_FJNbtagJets', 'm_FJphi', 
                'm_FJeta', 'm_DTeta', 'm_DTphi']

    ## Define input variables 
    for branch in backg_tree.GetListOfBranches():
        if branch.GetName() in not_cons:
            continue
        loader.AddVariable(branch.GetName())
        
    mycuts = ROOT.TCut("")   ## for example: TCut mycuts = "abs(var1)<0.5 && abs(var2-0.5)<1";
    mycutb = ROOT.TCut("")   ## for example: TCut mycutb = "abs(var1)<0.5";


    loader.PrepareTrainingAndTestTree(mycuts, mycutb, training)
    
    ## Boosted Decision Trees
    factory.BookMethod(loader,ROOT.TMVA.Types.kBDT, "BDT", params)
    
    #loader.OptimizeTuningParameters("ROCIntegral","FitGA")
    
    factory.TrainAllMethods()
    
    factory.TestAllMethods()
    
    factory.EvaluateAllMethods()
    
    #c1 = factory.GetROCCurve(loader)
    #c1.Draw()
    
    integ = factory.GetROCIntegral(loader, "BDT")
    
    imethod = factory.GetMethod("../bdt_output/dataset2", "BDT")
    #method = dynamic_cast<MethodBase *>(imethod);
    icut = imethod.GetSignalReferenceCut()
    print("cut:", icut)
    
    print("ROC integral:", integ)
    
    outputFile.Close()
    signal_input.Close()
    backg_input.Close()
    
    return integ

In [8]:
def write_combs_params(file_params, file_training, arr_NTrees, arr_MinNodeSize, arr_MaxDepth, arr_BoostType, arr_AdaBoostBeta, arr_BaggedSampleFraction, arr_SeparationType, arr_nCuts, arr_nTrain_Signal, arr_nTrain_Background):
    comb_params = list(itertools.product(arr_NTrees, arr_MinNodeSize, 
                                         arr_MaxDepth, arr_BoostType, 
                                         arr_AdaBoostBeta, 
                                         arr_BaggedSampleFraction, 
                                         arr_SeparationType, 
                                         arr_nCuts, arr_nTrain_Signal, 
                                         arr_nTrain_Background))
    
    with open(file_params, 'w') as params, open(file_training, 'w') as training:
        for cp in comb_params:
            string1 = "!V:NTrees="+str(cp[0])+":MinNodeSize="+str(cp[1])+"%:MaxDepth="+str(cp[2])+":BoostType="+cp[3]+":AdaBoostBeta="+str(cp[4])+":UseBaggedBoost:BaggedSampleFraction="+str(cp[5])+":SeparationType="+cp[6]+":nCuts="+str(cp[7])+"\n"
            params.write(string1)
            string2 = "nTrain_Signal="+str(cp[8])+":nTrain_Background="+str(cp[9])+":SplitMode=Random:NormMode=NumEvents:!V\n"
            training.write(string2)

In [9]:
file_params = outputDir+"bdt_params2.txt"
file_training = outputDir+"bdt_training2.txt"
arr_NTrees = [2000]
arr_MinNodeSize = [6]
arr_MaxDepth = [2]
arr_BoostType = ["AdaBoost", "Bagging"]#, AdaBoost,RealAdaBoost,Bagging,Grad
arr_AdaBoostBeta = [0.2]
arr_BaggedSampleFraction = [0.3]
arr_SeparationType = ["MisClassificationError", "CrossEntropy", "GiniIndex"] #  "MisClassificationError", "SDivSqrtSPlusB", "CrossEntropy", "GiniIndex"
arr_nCuts = [20]
arr_nTrain_Signal = [1000]
arr_nTrain_Background = [1000]

In [10]:
write_combs_params(file_params, file_training, arr_NTrees, arr_MinNodeSize, 
                   arr_MaxDepth, arr_BoostType, arr_AdaBoostBeta, 
                   arr_BaggedSampleFraction, arr_SeparationType, arr_nCuts, 
                   arr_nTrain_Signal, arr_nTrain_Background)

In [11]:
def param_opt(config, params, training,outputDir):
    max_roc = 0
    best_params = ""
    best_train = ""
    print(config)
    print("===============")
    for i in range(len(params)):
        roc = BDT(params[i], training[i], config, outputDir)
        if roc > max_roc:
            max_roc = roc
            best_params = params[i]
            best_train = training[i]
    print("best parameters:", best_params)
    print("best training:", best_train)
    print("ROC integral:", max_roc) 

In [12]:
params_path = outputDir+"bdt_params2.txt"
params = get_params(params_path)
training_path = outputDir+ "bdt_training2.txt"
training = get_params(training_path)
configs = ["SR_1_Xtohh1000_Hw", "SR_2_Xtohh2000_Hw"]
s_end = "_Hw_s.root"
b_end = "_Hw_b.root"

!V:NTrees=2000:MinNodeSize=6%:MaxDepth=2:BoostType=AdaBoost:AdaBoostBeta=0.2:UseBaggedBoost:BaggedSampleFraction=0.3:SeparationType=MisClassificationError:nCuts=20

!V:NTrees=2000:MinNodeSize=6%:MaxDepth=2:BoostType=AdaBoost:AdaBoostBeta=0.2:UseBaggedBoost:BaggedSampleFraction=0.3:SeparationType=CrossEntropy:nCuts=20

!V:NTrees=2000:MinNodeSize=6%:MaxDepth=2:BoostType=AdaBoost:AdaBoostBeta=0.2:UseBaggedBoost:BaggedSampleFraction=0.3:SeparationType=GiniIndex:nCuts=20

!V:NTrees=2000:MinNodeSize=6%:MaxDepth=2:BoostType=Bagging:AdaBoostBeta=0.2:UseBaggedBoost:BaggedSampleFraction=0.3:SeparationType=MisClassificationError:nCuts=20

!V:NTrees=2000:MinNodeSize=6%:MaxDepth=2:BoostType=Bagging:AdaBoostBeta=0.2:UseBaggedBoost:BaggedSampleFraction=0.3:SeparationType=CrossEntropy:nCuts=20

!V:NTrees=2000:MinNodeSize=6%:MaxDepth=2:BoostType=Bagging:AdaBoostBeta=0.2:UseBaggedBoost:BaggedSampleFraction=0.3:SeparationType=GiniIndex:nCuts=20

nTrain_Signal=1000:nTrain_Background=1000:SplitMode=Random:

In [13]:
configs[-2]

'SR_1_Xtohh1000_Hw'

In [14]:
params_ = "!V:NTrees=1000:MinNodeSize=4%:MaxDepth=4:BoostType=AdaBoost:AdaBoostBeta=0.2:UseBaggedBoost:BaggedSampleFraction=0.6:SeparationType=MisClassificationError:nCuts=20"
training_ = "nTrain_Signal=1000:nTrain_Background=1000:SplitMode=Random:NormMode=NumEvents:!V"
comb = "COMB4"
roc = BDT(params_, training_, configs[-1], outputDir, comb)

cut: 0.4601465914128369
ROC integral: 0.9899808381726068
                         : 
                         : Evaluation results ranked by best signal efficiency and purity (area)
                         : -------------------------------------------------------------------------------------------------------------------
                         : DataSet       MVA                       
                         : Name:         Method:          ROC-integ
                         : ../bdt_output/dataset2 BDT            : 0.990
                         : -------------------------------------------------------------------------------------------------------------------
                         : 
                         : Testing efficiency compared to training efficiency (overtraining check)
                         : -------------------------------------------------------------------------------------------------------------------
                         : DataSet              MVA  

In [15]:
# Trying BDT with different combinations of parameters
param_opt(configs[-2], params, training, outputDir)

SR_1_Xtohh1000_Hw
cut: 0.2798386832305853
ROC integral: 0.9577977765327697
cut: 0.2500683756560796
ROC integral: 0.9555249020578584
cut: 0.27523635224588566
ROC integral: 0.958611939836774
cut: -0.029989386995942197
ROC integral: 0.9620389976887924
cut: -0.10297312651556331
ROC integral: 0.9585202086193354
cut: -0.08590932299551415
ROC integral: 0.9605144143963098
best parameters: !V:NTrees=2000:MinNodeSize=6%:MaxDepth=2:BoostType=Bagging:AdaBoostBeta=0.2:UseBaggedBoost:BaggedSampleFraction=0.3:SeparationType=MisClassificationError:nCuts=20
best training: nTrain_Signal=1000:nTrain_Background=1000:SplitMode=Random:NormMode=NumEvents:!V
ROC integral: 0.9620389976887924
                         : 
                         : Evaluation results ranked by best signal efficiency and purity (area)
                         : -------------------------------------------------------------------------------------------------------------------
                         : DataSet       MVA            

Warning in <TFile::Init>: file ../bdt_output/SR_1_Xtohh1000_Hw_s.root probably not closed, trying to recover
Info in <TFile::Recover>: ../bdt_output/SR_1_Xtohh1000_Hw_s.root, recovered key TTree:Nominal at address 2921
Warning in <TFile::Init>: successfully recovered 1 keys
Warning in <TFile::Init>: file ../bdt_output/SR_1_Xtohh1000_Hw_b.root probably not closed, trying to recover
Info in <TFile::Recover>: ../bdt_output/SR_1_Xtohh1000_Hw_b.root, recovered key TTree:Nominal at address 10537
Warning in <TFile::Init>: successfully recovered 1 keys
Warning in <TFile::Init>: file ../bdt_output/SR_1_Xtohh1000_Hw_s.root probably not closed, trying to recover
Info in <TFile::Recover>: ../bdt_output/SR_1_Xtohh1000_Hw_s.root, recovered key TTree:Nominal at address 2921
Warning in <TFile::Init>: successfully recovered 1 keys
Warning in <TFile::Init>: file ../bdt_output/SR_1_Xtohh1000_Hw_b.root probably not closed, trying to recover
Info in <TFile::Recover>: ../bdt_output/SR_1_Xtohh1000_Hw_b.root,